In [ ]:
!pip install koreanize-matplotlib

In [ ]:
#CSV 파일을 읽어와 정보 출력하는 코드

from ipywidgets import FileUpload, Button, Output
from IPython.display import display
import pandas as pd
import io

# 파일 업로드 위젯 생성
upload = FileUpload(accept='.csv', multiple=False)
upload_button = Button(description='데이터 불러오기')
output_area = Output()

# 전역 변수 my_df 선언

my_df = None

# 업로드 후 CSV 파일을 읽어오기
def load_file(change):
    global my_df  # 전역 변수를 사용하여 df 저장
    output_area.clear_output()  # 출력 영역 초기화

    # 파일 내용 추출
    if len(upload.value) == 0:
        return

    # 업로드한 파일의 내용을 판다스로 읽어오기
    try:
        file_content = upload.value[list(upload.value.keys())[0]]['content']
        # 여러 인코딩 방식으로 파일을 읽기 시도
        try:
            my_df = pd.read_csv(io.BytesIO(file_content), encoding='utf-8')
        except UnicodeDecodeError:
            try:
                my_df = pd.read_csv(io.BytesIO(file_content), encoding='utf-8-sig')
            except UnicodeDecodeError:
                my_df = pd.read_csv(io.BytesIO(file_content), encoding='cp949')


        with output_area:
            display(my_df.info())
            display(my_df.head())
            display(my_df.describe())


    except Exception as e:
        with output_area:
            print(f"Error: {e}")

# 버튼 클릭 시 파일 로드 함수 실행
upload_button.on_click(load_file)

# 위젯 출력
display(upload)
display(upload_button)
display(output_area)



In [ ]:
#텍스트 속성 분석을 위해 데이터프레임에서 고유값을 확인하는 코드

from ipywidgets import Select, Button, Output, VBox
from IPython.display import display

# 전역 변수 my_df가 이미 있다고 가정하고 진행
# my_df는 CSV 데이터를 포함한 pandas DataFrame

# 목록 상자를 위한 위젯
select_attribute = Select(description='속성 선택:', options=[], disabled=True)
show_unique_button = Button(description='고유 값 보기', disabled=True)
output_area = Output()

# object 타입의 컬럼 추출하여 목록 상자에 표시
def setup_attribute_selection():
    object_columns = my_df.select_dtypes(include=['object']).columns.tolist()
    select_attribute.options = object_columns
    select_attribute.disabled = False
    show_unique_button.disabled = False

# 선택된 속성의 고유값을 출력하는 함수
def show_unique_values(change):
    selected_column = select_attribute.value
    unique_values = my_df[selected_column].unique()

    with output_area:
        output_area.clear_output()
        print(f"'{selected_column}'의 고유 값:")
        print(unique_values)

# 목록 상자에서 속성 선택 후 고유값 보기
show_unique_button.on_click(show_unique_values)

# 속성 선택 목록 상자를 세팅
setup_attribute_selection()

# 위젯 출력
display(VBox([select_attribute, show_unique_button, output_area]))


In [ ]:
import pandas as pd
from ipywidgets import Checkbox, Button, Output, VBox, HBox, Label
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import graphviz

# 체크박스를 통해 독립 변수와 종속 변수를 선택할 수 있도록 하기
independent_checkboxes = []
dependent_checkboxes = []

for col in my_df.columns:
    independent_checkboxes.append(Checkbox(value=False, description=col))
    dependent_checkboxes.append(Checkbox(value=False, description=col))

# 레이블 추가
independent_label = Label("독립 변수 선택:")
dependent_label = Label("종속 변수 선택:")

# 독립 변수와 종속 변수 박스
independent_box = VBox([independent_label] + independent_checkboxes)
dependent_box = VBox([dependent_label] + dependent_checkboxes)

# 버튼 생성
train_button = Button(description="트리 학습 및 저장")
output_area = Output()

def train_decision_tree(b):
    output_area.clear_output()

    # 독립 변수와 종속 변수 선택
    selected_independent = [cb.description for cb in independent_checkboxes if cb.value]
    selected_dependent = [cb.description for cb in dependent_checkboxes if cb.value]

    if len(selected_dependent) != 1:
        with output_area:
            print("하나의 종속 변수를 선택하세요.")
        return

    X = my_df[selected_independent].copy()
    y = my_df[selected_dependent[0]].copy()

    # 레이블 인코더를 저장할 딕셔너리 생성
    encoders = {}

    # 독립 변수의 범주형 데이터 레이블 인코딩
    X_encoded = X.copy()
    for col in X.columns:
        if X[col].dtype == 'object':
            le = LabelEncoder()
            X_encoded[col] = le.fit_transform(X[col])
            encoders[col] = le  # 인코더 저장

    # 종속 변수 y도 범주형이면 레이블 인코딩
    if y.dtype == 'object':
        y_le = LabelEncoder()
        y = y_le.fit_transform(y)
        class_names = y_le.classes_
    else:
        class_names = [str(cls) for cls in y.unique()]

    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

    # 의사 결정 트리 모델 학습
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)

    # Graphviz로 트리 시각화 및 PDF 저장
    dot_data = export_graphviz(clf, out_file=None,
                               feature_names=X_encoded.columns,
                               class_names=[str(cls) for cls in class_names],
                               filled=True,
                               special_characters=True,
                               fontname="NanumGothic")
    graph = graphviz.Source(dot_data)
    graph.render("decision_tree")  # decision_tree.pdf로 저장됨

    with output_area:
        print("트리가 학습되었습니다. 'decision_tree.pdf'로 저장되었습니다.")

    # 오차 행렬 출력
    y_pred = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    # 오차 행렬을 시각적으로 출력
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    # 성능 지표 계산
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    with output_area:
        disp.plot()
        print(f"정확도(Accuracy): {accuracy:.2f} - 전체 데이터 중 맞게 예측한 비율")
        print(f"정밀도(Precision): {precision:.2f} - 양성 예측 중 실제 양성인 비율")
        print(f"재현율(Recall): {recall:.2f} - 실제 양성 중 양성으로 예측한 비율")
        print(f"F1 스코어(F1-Score): {f1:.2f} - 정밀도와 재현율의 조화 평균")

        # 독립 변수의 레이블 매핑 출력
        if encoders:
            print("\n독립 변수의 레이블 매핑:")
            for col, le in encoders.items():
                labels = le.classes_
                mapping = {index: label for index, label in enumerate(labels)}
                print(f"'{col}' 변수:")
                for index, label in mapping.items():
                    print(f"  {index}: {label}")
                print()

        # 종속 변수의 레이블 매핑 출력
        if 'y_le' in locals():
            label_mapping = {index: label for index, label in enumerate(y_le.classes_)}
            print("종속 변수의 레이블 매핑:")
            for index, label in label_mapping.items():
                print(f"  {index}: {label}")
            print()

# 버튼 클릭 시 트리 학습 함수 호출
train_button.on_click(train_decision_tree)

# 위젯 표시 (독립 변수는 좌측, 종속 변수는 우측에 배치)
display(HBox([independent_box, dependent_box]))
display(train_button)
display(output_area)


In [ ]:
import pandas as pd
from ipywidgets import Checkbox, Button, Output, VBox, HBox, Label, FileUpload
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import io
from IPython.display import display

# 체크박스를 통해 독립 변수와 종속 변수를 선택할 수 있도록 하기
independent_checkboxes = []
dependent_checkboxes = []

for col in my_df.columns:
    independent_checkboxes.append(Checkbox(value=False, description=col))
    dependent_checkboxes.append(Checkbox(value=False, description=col))

# 레이블 추가
independent_label = Label("독립 변수 선택:")
dependent_label = Label("종속 변수 선택:")

# 독립 변수와 종속 변수 박스
independent_box = VBox([independent_label] + independent_checkboxes)
dependent_box = VBox([dependent_label] + dependent_checkboxes)

# 버튼 생성
train_button = Button(description="랜덤 포레스트 학습")
output_area = Output()

# 테스트 파일 업로드 위젯 및 버튼 생성
test_upload = FileUpload(accept='.csv', multiple=False)
test_button = Button(description="테스트 데이터 적용")
save_button = Button(description="TEST 데이터 CSV 생성", disabled=True)

# 레이블 인코더를 저장할 딕셔너리 생성
encoders = {}
model = None  # 전역 변수로 모델 선언

def train_random_forest(b):
    global model, encoders, class_names, y_le
    output_area.clear_output()

    # 독립 변수와 종속 변수 선택
    selected_independent = [cb.description for cb in independent_checkboxes if cb.value]
    selected_dependent = [cb.description for cb in dependent_checkboxes if cb.value]

    if len(selected_dependent) != 1:
        with output_area:
            print("하나의 종속 변수를 선택하세요.")
        return

    X = my_df[selected_independent].copy()
    y = my_df[selected_dependent[0]].copy()

    # 레이블 인코더를 저장할 딕셔너리 초기화
    encoders = {}

    # 독립 변수의 범주형 데이터 레이블 인코딩
    X_encoded = X.copy()
    for col in X.columns:
        if X[col].dtype == 'object':
            le = LabelEncoder()
            X_encoded[col] = le.fit_transform(X[col])
            encoders[col] = le  # 인코더 저장

    # 종속 변수 y도 범주형이면 레이블 인코딩
    if y.dtype == 'object':
        y_le = LabelEncoder()
        y = y_le.fit_transform(y)
        class_names = y_le.classes_
    else:
        class_names = [str(cls) for cls in y.unique()]
        y_le = None

    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

    # 랜덤 포레스트 모델 학습
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    with output_area:
        print("랜덤 포레스트 모델이 학습되었습니다.")

    # 오차 행렬 출력
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    # 오차 행렬을 시각적으로 출력
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    # 성능 지표 계산
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    with output_area:
        disp.plot()
        print(f"정확도(Accuracy): {accuracy:.2f} - 전체 데이터 중 맞게 예측한 비율")
        print(f"정밀도(Precision): {precision:.2f} - 양성 예측 중 실제 양성인 비율")
        print(f"재현율(Recall): {recall:.2f} - 실제 양성 중 양성으로 예측한 비율")
        print(f"F1 스코어(F1-Score): {f1:.2f} - 정밀도와 재현율의 조화 평균")

        # 독립 변수의 레이블 매핑 출력
        if encoders:
            print("\n독립 변수의 레이블 매핑:")
            for col, le in encoders.items():
                labels = le.classes_
                mapping = {index: label for index, label in enumerate(labels)}
                print(f"'{col}' 변수:")
                for index, label in mapping.items():
                    print(f"  {index}: {label}")
                print()

        # 종속 변수의 레이블 매핑 출력
        if y_le is not None:
            label_mapping = {index: label for index, label in enumerate(y_le.classes_)}
            print("종속 변수의 레이블 매핑:")
            for index, label in label_mapping.items():
                print(f"  {index}: {label}")
            print()

def apply_test_data(b):
    global test_df
    output_area.clear_output()

    if model is None:
        with output_area:
            print("먼저 모델을 학습시켜주세요.")
        return

    # 테스트 파일 내용 추출
    if len(test_upload.value) == 0:
        with output_area:
            print("테스트 데이터를 업로드해주세요.")
        return

    # 업로드한 파일의 내용을 판다스로 읽어오기
    try:
        file_content = test_upload.value[list(test_upload.value.keys())[0]]['content']
        # 여러 인코딩 방식으로 파일을 읽기 시도
        try:
            test_df = pd.read_csv(io.BytesIO(file_content), encoding='utf-8')
        except UnicodeDecodeError:
            try:
                test_df = pd.read_csv(io.BytesIO(file_content), encoding='utf-8-sig')
            except UnicodeDecodeError:
                test_df = pd.read_csv(io.BytesIO(file_content), encoding='cp949')

        # 독립 변수 선택
        selected_independent = [cb.description for cb in independent_checkboxes if cb.value]
        selected_dependent = [cb.description for cb in dependent_checkboxes if cb.value]

        # 테스트 데이터에 필요한 독립 변수가 모두 있는지 확인
        missing_features = set(selected_independent) - set(test_df.columns)
        if missing_features:
            with output_area:
                print(f"테스트 데이터에 필요한 속성이 없습니다: {', '.join(missing_features)}")
                print("파일을 다시 업로드 해주세요.")
            return

        X_test = test_df[selected_independent].copy()

        # 독립 변수의 범주형 데이터 레이블 인코딩 (학습 시 사용한 인코더로)
        X_test_encoded = X_test.copy()
        for col in X_test.columns:
            if X_test[col].dtype == 'object':
                if col in encoders:
                    le = encoders[col]
                    # 테스트 데이터에 학습 때 없었던 새로운 범주가 있는지 확인
                    unseen_labels = set(X_test[col].unique()) - set(le.classes_)
                    if unseen_labels:
                        with output_area:
                            print(f"테스트 데이터의 '{col}' 변수에 학습되지 않은 새로운 범주가 있습니다: {', '.join(unseen_labels)}")
                            print("파일을 다시 업로드 해주세요.")
                        return
                    X_test_encoded[col] = le.transform(X_test[col])
                else:
                    with output_area:
                        print(f"변수 '{col}'에 대한 인코더가 없습니다.")
                    return

        # 결측치가 있는 행 처리
        missing_value_rows = X_test_encoded.isnull().any(axis=1)
        excluded_records = X_test[missing_value_rows]
        X_test_encoded = X_test_encoded[~missing_value_rows]

        # 예측 수행
        y_pred = model.predict(X_test_encoded)

        # 예측 결과를 원래 클래스 이름으로 변환
        if y_le is not None:
            y_pred_labels = y_le.inverse_transform(y_pred)
        else:
            y_pred_labels = y_pred

        # 예측 결과를 테스트 데이터에 추가
        test_df.loc[~missing_value_rows, 'Prediction'] = y_pred_labels

        # 결측치가 있는 레코드 처리
        if missing_value_rows.any():
            with output_area:
                print("다음 레코드는 속성 값이 누락되어 예측에서 제외되었습니다:")
                for idx in excluded_records.index:
                    missing_cols = X_test.columns[X_test.loc[idx].isnull()]
                    print(f"인덱스 {idx}: 누락된 속성 - {', '.join(missing_cols)}")
                    print("해당 레코드의 속성 값:")
                    display(test_df.loc[idx, selected_independent])
                    print()
        else:
            with output_area:
                print("모든 레코드에 대해 예측이 수행되었습니다.")

        # 종속 변수 실제값이 있다면 가져오기
        if selected_dependent[0] in test_df.columns:
            y_test = test_df.loc[~missing_value_rows, selected_dependent[0]].copy()
            # 종속 변수 y도 범주형이면 레이블 인코딩 (학습 시 사용한 인코더로)
            if y_test.dtype == 'object':
                if y_le is not None:
                    y_test = y_le.transform(y_test)
                else:
                    with output_area:
                        print("종속 변수의 인코더가 없습니다.")
                        return
        else:
            y_test = None

        # 성능 평가 (y_test가 있는 경우)
        if y_test is not None:
            y_pred_used = y_pred  # 이미 결측치 제거된 데이터에 대한 예측 결과
            cm = confusion_matrix(y_test, y_pred_used)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

            accuracy = accuracy_score(y_test, y_pred_used)
            precision = precision_score(y_test, y_pred_used, average='weighted', zero_division=0)
            recall = recall_score(y_test, y_pred_used, average='weighted', zero_division=0)
            f1 = f1_score(y_test, y_pred_used, average='weighted', zero_division=0)

            with output_area:
                disp.plot()
                print(f"정확도(Accuracy): {accuracy:.2f} - 전체 데이터 중 맞게 예측한 비율")
                print(f"정밀도(Precision): {precision:.2f} - 양성 예측 중 실제 양성인 비율")
                print(f"재현율(Recall): {recall:.2f} - 실제 양성 중 양성으로 예측한 비율")
                print(f"F1 스코어(F1-Score): {f1:.2f} - 정밀도와 재현율의 조화 평균")
        else:
            with output_area:
                print("테스트 데이터에 실제 종속 변수 값이 없어 성능 평가를 할 수 없습니다.")

        # 저장 버튼 활성화
        save_button.disabled = False

    except Exception as e:
        with output_area:
            print(f"Error: {e}")

def save_test_results(b):
    output_area.clear_output()

    if 'test_df' not in globals():
        with output_area:
            print("테스트 데이터가 없습니다.")
        return

    # 테스트 데이터와 예측 결과를 CSV로 저장
    try:
        test_df.to_csv('test_results.csv', index=False, encoding='utf-8-sig')
        with output_area:
            print("테스트 데이터에 예측 결과를 추가하여 'test_results.csv'로 저장하였습니다.")
    except Exception as e:
        with output_area:
            print(f"Error: {e}")

# 버튼 클릭 시 함수 호출
train_button.on_click(train_random_forest)
test_button.on_click(apply_test_data)
save_button.on_click(save_test_results)

# 위젯 표시
display(HBox([independent_box, dependent_box]))
display(train_button)
display(HBox([Label("테스트 데이터 업로드:"), test_upload, test_button, save_button]))
display(output_area)
